In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = dt.date(2022, 2, 8) ##for exec board (prior run for comparison)
itp_id = 300

In [3]:
bbb = rt.OperatorDayAnalysis(itp_id, analysis_date)

found parquet
found parquet
found parquet
found parquet
found parquet


In [16]:
interp = bbb.position_interpolators['860682']['rt']

In [17]:
interp.cleaned_positions.shape

(41, 12)

In [18]:
interp.position_gdf.shape

(66, 12)

In [19]:
# interp.detailed_speed_map()

In [23]:
test = interp.position_gdf.iloc[60:,:]

In [26]:
a = test.shape_meters.to_numpy()

In [27]:
a

array([11507.03032107, 11761.01140182, 11927.73109127, 11926.6027973 ,
       12932.70205404, 13148.18336111])

In [28]:
import numpy as np

In [31]:
from numba import jit

In [32]:
%%timeit
np.maximum.accumulate(a)

1.9 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [35]:
@jit(nopython=True)
def numba_accumulate(b):
    for i in range(1, b.size):
        if b[i] < b[i-1]:
            b[i] = b[i-1]
    return b
    # return np.maximum.accumulate(a)

In [38]:
%%timeit
numba_accumulate(a)

2.41 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [39]:
numba_accumulate(a)

array([11507.03032107, 11761.01140182, 11927.73109127, 11927.73109127,
       12932.70205404, 13148.18336111])

In [40]:
np.maximum.accumulate(a)

array([11507.03032107, 11761.01140182, 11927.73109127, 11927.73109127,
       12932.70205404, 13148.18336111])

In [71]:
#     def _linear_reference(self):
        
#         self.position_gdf = self._shift_calculate(self.position_gdf)
#         self.progressing_positions = self.position_gdf >> filter(_.progressed)
#         self.debug_dict['clean_0'] = self.progressing_positions.copy()
#         ## check if positions have progressed from immediate previous point, but not previous point of forwards progression
#         while not self.progressing_positions.shape_meters.is_monotonic:
#             self._position_cleaning_count += 1
#             # print(f'check location data for trip {self.trip_id}')
#             self.progressing_positions = self._shift_calculate(self.progressing_positions)
#             self.progressing_positions = self.progressing_positions >> filter(_.progressed)
#             self.debug_dict[f'clean_{self._position_cleaning_count}'] = self.progressing_positions.copy()
#         self.cleaned_positions = self.progressing_positions ## for position and map methods in TripPositionInterpolator
#         self.cleaned_positions = self.cleaned_positions >> arrange(self.time_col)
#         self._shape_array = self.cleaned_positions.shape_meters.to_numpy()
#         self._dt_array = (self.cleaned_positions[self.time_col].to_numpy()
#                                       .astype('datetime64[s]')
#                                       .astype('float64')
#                                      )

In [50]:
def lin_ref_experimental(vp_interp):
    raw_positions = vp_interp.position_gdf
    raw_positions = raw_positions >> arrange(vp_interp.time_col)
    raw_position_array = raw_positions.shape_meters.to_numpy()
    cast_monotonic = np.maximum.accumulate(raw_position_array)
    raw_positions.shape_meters = cast_monotonic
    raw_positions = raw_positions.drop_duplicates(subset=['shape_meters'], keep = 'last')
    raw_positions['secs_from_last'] = raw_positions[vp_interp.time_col].diff()
    raw_positions.secs_from_last = (raw_positions.secs_from_last
                                    .apply(lambda x: x.seconds))
    raw_positions['meters_from_last'] = raw_positions.shape_meters.diff()
    raw_positions['speed_from_last'] = (raw_positions.meters_from_last
                                                 / raw_positions.secs_from_last) ## meters/second
    vp_interp.cleaned_positions = raw_positions
    vp_interp._shape_array = vp_interp.cleaned_positions.shape_meters.to_numpy()
    vp_interp._dt_array = (vp_interp.cleaned_positions[vp_interp.time_col].to_numpy()
                                      .astype('datetime64[s]')
                                      .astype('float64')
                                     )
    return

In [72]:
%%timeit
interp._linear_reference()

31.3 ms ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
%%timeit
lin_ref_experimental(interp)

22.8 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
new.shape

(41, 12)

In [61]:
old.shape

(41, 12)

In [70]:
new.shape_meters.reset_index(drop=True) == old.shape_meters.reset_index(drop=True)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
30    True
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
39    True
40    True
Name: shape_meters, dtype: bool